# Stock Project - Real-Time Data Pipeline


#### Get stock data

In [ ]:
# Create default deployment stage parameter
deployment_stage_parameters = "DEV" 

In [ ]:
# Use json file saved in DEV lakehouse as workaround to get deployment stage as parameter

import json

config_dict = json.loads(deployment_stage_parameters)

env = config_dict.get("environment", {})
default_params = {
    # Extract individual values safely
    "stage": env.get("Stage", "")
}
stage = default_params["stage"]
sql = f"SELECT '{stage}' AS Stage"

mssparkutils.notebook.exit(sql)

In [2]:
import yfinance as yf 
import pandas as pd 
from pyspark.sql.functions import col

# List of tickers you want to download
all_tickers = ["AAPL", "MSFT", "GOOGL", "AMZN", "TSLA"]

# Stage-specific selection
if stage == "DEV":
    tickers = ["AAPL", "MSFT"]
else: 
    tickers = all_tickers


# Download 1-minute interval data for all tickers for the last trading day
# yfinance returns a multi-index DataFrame when multiple tickers are used
df = yf.download(
    tickers=tickers,
    interval="1m",
    period="1d",
    group_by="ticker"  # Organize data by ticker symbol
)

# Show the first rows
df.head()


StatementMeta(, d50b9644-0988-400c-aa81-ca4582bef9a1, 19, Finished, Available, Finished)

[*******************   40%                       ]  2 of 5 completed

[*********************100%***********************]  5 of 5 completed


Ticker                           AAPL                                      \
Price                            Open        High         Low       Close   
Datetime                                                                    
2026-01-14 14:30:00+00:00  259.480011  259.670013  259.244995  259.640015   
2026-01-14 14:31:00+00:00  259.600098  259.850006  259.239990  259.700012   
2026-01-14 14:32:00+00:00  259.839996  261.140015  259.839996  261.029999   
2026-01-14 14:33:00+00:00  261.029999  261.320007  260.739990  261.320007   
2026-01-14 14:34:00+00:00  261.290009  261.809998  261.260010  261.690002   

Ticker                                      MSFT                          \
Price                         Volume        Open        High         Low   
Datetime                                                                   
2026-01-14 14:30:00+00:00  1064294.0  467.334991  467.750000  467.000000   
2026-01-14 14:31:00+00:00   161857.0  467.000000  467.269897  466.410004   
2026-01-14 14:32:00+00:00   138030.0  466.619904  467.700012  466.030090   
2026-01-14 14:33:00+00:00   101261.0  467.209991  468.070007  467.138489   
2026-01-14 14:34:00+00:00   163534.0  467.579987  468.170013  467.390015   

Ticker                                           ...       GOOGL              \
Price                           Close    Volume  ...        Open        High   
Datetime                                         ...                           
2026-01-14 14:30:00+00:00  467.000000  637247.0  ...  333.595093  334.299896   
2026-01-14 14:31:00+00:00  466.529999  113352.0  ...  334.299988  334.359985   
2026-01-14 14:32:00+00:00  467.209991  145499.0  ...  332.600006  332.910004   
2026-01-14 14:33:00+00:00  467.521790   57920.0  ...  332.829987  333.959991   
2026-01-14 14:34:00+00:00  467.695007   70312.0  ...  333.649994  334.089996   

Ticker                                                            TSLA  \
Price                             Low       Close   Volume        Open   
Datetime                                                                 
2026-01-14 14:30:00+00:00  333.579987  334.190002  1145196  439.665009   
2026-01-14 14:31:00+00:00  332.279999  332.609985   204333  439.880005   
2026-01-14 14:32:00+00:00  332.049988  332.769989   153675  439.174988   
2026-01-14 14:33:00+00:00  332.779999  333.587494   110796  440.299988   
2026-01-14 14:34:00+00:00  333.320007  333.700012   136668  440.649994   

Ticker                                                                    
Price                            High         Low       Close     Volume  
Datetime                                                                  
2026-01-14 14:30:00+00:00  439.929901  439.290009  439.850006  1160455.0  
2026-01-14 14:31:00+00:00  439.929993  439.040009  439.204987   281200.0  
2026-01-14 14:32:00+00:00  440.757507  439.010010  440.343414   310536.0  
2026-01-14 14:33:00+00:00  441.279999  440.149994  440.630005   281171.0  
2026-01-14 14:34:00+00:00  441.700012  439.880005  439.890015   247316.0  

[5 rows x 25 columns]

#### Flatten the multi-index DataFrame returned by yfinance

In [3]:
# Flatten the multi-index DataFrame returned by yfinance
# This creates a clean table with columns: ticker, datetime, open, high, low, close, volume
rows = []

# Loop through each ticker and extract its data
for ticker in tickers:
    # Extract the sub-DataFrame for the ticker
    temp = df[ticker].copy()
    temp = temp.reset_index()
    
    # Add the ticker column
    temp["ticker"] = ticker
    
    # Append to the list
    rows.append(temp)

# Combine all tickers into one DataFrame
df_flat = pd.concat(rows, ignore_index=True)

# Show the first rows
df_flat.head()


StatementMeta(, d50b9644-0988-400c-aa81-ca4582bef9a1, 20, Finished, Available, Finished)

Price,Datetime,Open,High,Low,Close,Volume,ticker
0,2026-01-14 14:30:00+00:00,259.480011,259.670013,259.244995,259.640015,1064294.0,AAPL
1,2026-01-14 14:31:00+00:00,259.600098,259.850006,259.239990,259.700012,161857.0,AAPL
2,2026-01-14 14:32:00+00:00,259.839996,261.140015,259.839996,261.029999,138030.0,AAPL
3,2026-01-14 14:33:00+00:00,261.029999,261.320007,260.739990,261.320007,101261.0,AAPL
4,2026-01-14 14:34:00+00:00,261.290009,261.809998,261.260010,261.690002,163534.0,AAPL


#### Write the Spark DataFrame to a Delta table in the connected Lakehouse

In [4]:
# Convert pandas DataFrame to Spark DataFrame
df_spark = spark.createDataFrame(df_flat)

from delta.tables import DeltaTable
from pyspark.sql.functions import col

# Ensure correct data types
df_spark = df_spark.withColumn("Datetime", col("Datetime").cast("timestamp"))
df_spark = df_spark.withColumn("ticker", col("ticker").cast("string"))

target_table = "bronze_stock_minutes"

# Check if the table already exists
table_exists = (
    spark.sql("SHOW TABLES")
    .filter("tableName = 'bronze_stock_minutes'")
    .count()
)

if table_exists == 0:
    # First run → create the table
    print("Creating new Delta table 'stock_prices'...")
    df_spark.write.format("delta").saveAsTable(target_table)

else:
    # Table exists → perform MERGE (upsert)
    print("Merging new data into existing Delta table...")

    delta_table = DeltaTable.forName(spark, target_table)

    (
        delta_table.alias("t")
        .merge(
            df_spark.alias("s"),
            "t.ticker = s.ticker AND t.Datetime = s.Datetime"
        )
        .whenNotMatchedInsertAll()
        .execute()
    )

print("MERGE completed successfully.")


StatementMeta(, d50b9644-0988-400c-aa81-ca4582bef9a1, 21, Finished, Available, Finished)

Merging new data into existing Delta table...


MERGE completed successfully.
